In [1]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam

In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from tensorflow import keras

In [3]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

170500096/170498071 [==============================] - 2s 0us/step


In [4]:
print(X_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [5]:
print(X_test.shape)
print(y_test.shape)

(10000, 32, 32, 3)
(10000, 1)


In [6]:
num_classes = 10
# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [7]:
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [8]:
X_train.shape

(50000, 32, 32, 3)

In [9]:
import numpy as np
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_mean

array([125.30691805, 122.95039414, 113.86538318])

In [10]:
X_train_std = np.std(X_train, axis=(0,1,2))
X_train_std

array([62.99321928, 62.08870764, 66.70489964])

In [11]:
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [12]:
#data augmentation
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) #we should not augment test data 

In [13]:
train_generator=train_datagen.flow(X_train,y_train,batch_size=150)
test_generator=test_datagen.flow(X_test,y_test,batch_size=150)

#callback when val accuracy reaches 90%

In [14]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > 0.90):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(0.90*100))   
            self.model.stop_training = True

In [15]:
val_acc_callback=myCallback()

#modelcheckpoint callback

In [16]:
filepath='/content/best_model_h5'

In [17]:
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
best_model_weight=tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True,save_weights_only=True, mode='auto', save_freq='epoch')

#checking how many parametrs are there

In [18]:
def no_of_connection(l):
    a=(l * (l+ 1))/2 # each layer has connection to its preceding and subsequent layer directly
    return a

def total_parametres(l, no_of_filters,no_of_dense_blocks,no_of_transition_blocks):
    input_layer_param= 3 * 3 * 3 * no_of_filters 
    para_dense_block= no_of_dense_blocks * ((no_of_filters * 3 * 3 * no_of_filters * no_of_connection(l)) + ( 4 * no_of_filters * no_of_connection(l)))
    para_transition_block=no_of_transition_blocks * ( (1 * 1 * no_of_filters * no_of_filters* ((l +1)) + 4 * no_of_filters * (l+1)))
    output_layer_params=((2 * 2 * no_of_filters * (l+1) * 10) + 10)  + (4 * no_of_filters * (l+1))

    return input_layer_param,para_dense_block,para_transition_block,output_layer_params

In [19]:
sum(total_parametres(13,17,4,3))
#we want less than 1 miliion parameter so we got here 

997451.0

#model

In [20]:
compression = 1
num_filter = 17
dropout_rate = 0
l = 13

In [21]:
# Dense Block
def denseblock(input, num_filter = 17, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 17, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [22]:

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [23]:
model = Model(inputs=[input], outputs=[output])

In [24]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 17)   459         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 17)   68          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 17)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit(train_generator,
                    steps_per_epoch=len(X_train)/150,
                    epochs=300,
                    verbose=1, 
                    validation_data=test_generator,
                    validation_steps=len(X_test)/150,
                    callbacks=[val_acc_callback,best_model_weight])

Epoch 1/300
334/333 [==============================] - 54s 160ms/step - loss: 1.7505 - accuracy: 0.3555 - val_loss: 3.6620 - val_accuracy: 0.1000
Epoch 2/300
334/333 [==============================] - 51s 154ms/step - loss: 1.4090 - accuracy: 0.4871 - val_loss: 1.6579 - val_accuracy: 0.4158
Epoch 3/300
334/333 [==============================] - 51s 154ms/step - loss: 1.2277 - accuracy: 0.5607 - val_loss: 1.5663 - val_accuracy: 0.5217
Epoch 4/300
334/333 [==============================] - 51s 154ms/step - loss: 1.1035 - accuracy: 0.6081 - val_loss: 1.5286 - val_accuracy: 0.5572
Epoch 5/300
334/333 [==============================] - 51s 154ms/step - loss: 1.0081 - accuracy: 0.6391 - val_loss: 1.1640 - val_accuracy: 0.6104
Epoch 6/300
334/333 [==============================] - 51s 153ms/step - loss: 0.9411 - accuracy: 0.6698 - val_loss: 1.4103 - val_accuracy: 0.5715
Epoch 7/300
334/333 [==============================] - 51s 154ms/step - loss: 0.8861 - accuracy: 0.6867 - val_loss: 1.3733 -

In [27]:
# Test the model
score = model.evaluate(test_generator, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

67/67 [==============================] - 3s 37ms/step - loss: 0.3415 - accuracy: 0.9030
Test loss: 0.3415147662162781
Test accuracy: 0.902999997138977


In [28]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


#conclusion:

* in densenet each layer is connected to its preceding layer and subsequent layer directly 

* densent is parameter efficient as compared other network like resnet , vgg, alexnet

* here i have created function fo checking how maany parametres does it contain so we can manage keep it below 1 million
